In [3]:
#QUENTIN_GROSHENS_2017#

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [21]:
def number_lines(filename,N_chunk=10000):
    chunks=pd.read_csv(filename,sep='^',chunksize=N_chunk) #cutting file in chuncks to stay in memory
    n_lines=0
    for chunk in chunks:
        n_lines=n_lines+chunk.shape[0]
    return n_lines

In [22]:
number_lines("searches.csv")

20390198

In [23]:
number_lines("bookings.csv")

10000010